In [1]:
%%bash
echo "TODAY'S DATE:"
date
echo "------------"
echo ""
#Display operating system info
lsb_release -a
echo ""
echo "------------"
echo "HOSTNAME: "; hostname 
echo ""
echo "------------"
echo "Computer Specs:"
echo ""
lscpu
echo ""
echo "------------"
echo ""
echo "Memory Specs"
echo ""
free -mh

TODAY'S DATE:
Wed Nov  6 14:57:08 PST 2019
------------

Distributor ID:	Ubuntu
Description:	Ubuntu 16.04.6 LTS
Release:	16.04
Codename:	xenial

------------
HOSTNAME: 
swoose

------------
Computer Specs:

Architecture:          x86_64
CPU op-mode(s):        32-bit, 64-bit
Byte Order:            Little Endian
CPU(s):                24
On-line CPU(s) list:   0-23
Thread(s) per core:    2
Core(s) per socket:    6
Socket(s):             2
NUMA node(s):          1
Vendor ID:             GenuineIntel
CPU family:            6
Model:                 44
Model name:            Intel(R) Xeon(R) CPU           X5670  @ 2.93GHz
Stepping:              2
CPU MHz:               2925.971
BogoMIPS:              5851.97
Virtualization:        VT-x
L1d cache:             32K
L1i cache:             32K
L2 cache:              256K
L3 cache:              12288K
NUMA node0 CPU(s):     0-23
Flags:                 fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush dts acpi mmx fxsr

No LSB modules are available.


### Set variables

`%env` are best for bash

In [2]:
# Set workding directory
%env wd=/home/sam/analyses/20191106_swoose_pgen_feature_stats
wd="/home/sam/analyses/20191106_swoose_pgen_feature_stats"

# File download commands
%env rsync_gannet=gannet:/volume2/web/Atumefaciens/20191105_swoose_pgen_v074_renaming/
%env wget_command=--directory-prefix=${wd} --quiety --no-directories --no-check-certificate  https://gannet.fish.washington.edu/Atumefaciens/20191105_swoose_pgen_v074_renaming/

# Input/output files    
%env gene_gff=Panopea-generosa-v1.0.a4.gene.gff3
%env gene_bed=Panopea-generosa-v1.0.a4.gene.bed
%env introns_bed=Panopea-generosa-v1.0.a4.introns.bed

# Programs
%env intersectbed=/home/sam/programs/bedtools-2.28.0/bin/intersectBed


env: wd=/home/sam/analyses/20191106_swoose_pgen_feature_stats
env: rsync_gannet=gannet:/volume2/web/Atumefaciens/20191105_swoose_pgen_v074_renaming/
env: wget_command=--directory-prefix=$/home/sam/analyses/20191106_swoose_pgen_feature_stats --quiety --no-directories --no-check-certificate  https://gannet.fish.washington.edu/Atumefaciens/20191105_swoose_pgen_v074_renaming/
env: gene_gff=Panopea-generosa-v1.0.a4.gene.gff3
env: gene_bed=Panopea-generosa-v1.0.a4.gene.bed
env: introns_bed=Panopea-generosa-v1.0.a4.introns.bed
env: intersectbed=/home/sam/programs/bedtools-2.28.0/bin/intersectBed


In [3]:
%%bash
mkdir --parents ${wd}

In [4]:
cd {wd}

/home/sam/analyses/20191106_swoose_pgen_feature_stats


### Download files

In [6]:
%%bash
# Create array of files from list
files_array=(${gene_gff} ${introns_bed})

for file in "${files_array[@]}"
do
  rsync \
  --archive \
  --progress \
  --verbose \
  "${rsync_gannet}${file}" \
  .
done

ls -lh

# If need to download via wget, uncomment lines in the cell below
# for file in "${files_array[@]}"
# do 
#  "${wget_command}${file}"
# done
# ls -lh ${wd}

receiving incremental file list
Panopea-generosa-v1.0.a4.gene.gff3
      9,888,459 100%   80.60MB/s    0:00:00 (xfr#1, to-chk=0/1)

sent 30 bytes  received 9,889,785 bytes  6,593,210.00 bytes/sec
total size is 9,888,459  speedup is 1.00
receiving incremental file list
Panopea-generosa-v1.0.a4.introns.bed
      4,579,012 100%   44.11MB/s    0:00:00 (xfr#1, to-chk=0/1)

sent 30 bytes  received 4,579,691 bytes  9,159,442.00 bytes/sec
total size is 4,579,012  speedup is 1.00
total 14M
-rw-rw-r-- 1 sam users 9.5M Nov  5 08:55 Panopea-generosa-v1.0.a4.gene.gff3
-rw-rw-r-- 1 sam users 4.4M Nov  5 08:59 Panopea-generosa-v1.0.a4.introns.bed


### Create gene BED file

In [7]:
%%bash
echo "Preview GFF: ${gene_gff}"
echo ""
head "${gene_gff}"
echo ""
echo "---------------------"
echo ""

# Create BED file
# Skip first three lines (header) of GFF
# Print scaffold name, start, and stop positions; tab-delimited
awk 'NR > 3 {print $1"\t"$4"\t"$5}' "${gene_gff}" \
> "${gene_bed}"

echo "Preview BED: ${gene_bed}"
echo ""
head "${gene_bed}"
echo ""

Preview GFF: Panopea-generosa-v1.0.a4.gene.gff3

##gff-version 3
##Generated using GenSAS, Monday 7th of October 2019 04:54:37 AM
##Project Name : Pgenerosa_v074
Scaffold_01	GenSAS_5d9637f372b5d-publish	gene	2	4719	.	+	.	ID=PGEN_.00g000010;Name=PGEN_.00g000010;original_ID=21510-PGEN_.00g234140;Alias=21510-PGEN_.00g234140;original_name=21510-PGEN_.00g234140;Notes=sp|Q86IC9|CAMT1_DICDI [BLAST protein vs protein (blastp) 2.7.1],PF01596.12 [Pfam 1.6]
Scaffold_01	GenSAS_5d9637f372b5d-publish	gene	19808	36739	.	-	.	ID=PGEN_.00g000020;Name=PGEN_.00g000020;original_ID=21510-PGEN_.00g234150;Alias=21510-PGEN_.00g234150;original_name=21510-PGEN_.00g234150;Notes=sp|P04177|TY3H_RAT [BLAST protein vs protein (blastp) 2.7.1],sp|P04177|TY3H_RAT [DIAMOND Functional 0.9.22],IPR036951 [InterProScan 5.29-68.0],PF00351.16 [Pfam 1.6]
Scaffold_01	GenSAS_5d9637f372b5d-publish	gene	49248	52578	.	-	.	ID=PGEN_.00g000030;Name=PGEN_.00g000030;original_ID=21510-PGEN_.00g234160;Alias=21510-PGEN_.00g234160;original_n

### Determine percent of genes containing introns

In [8]:
%%bash
# Count the number of genes
num_genes=$(cat ${gene_bed} | wc -l)
echo "${num_genes} found in ${gene_bed}"
echo ""
echo "----------------------------"
echo ""

# Determine number of genes with no intron overlap (that's the -v option below)
num_genes_no_introns=$(${intersectbed} -v -a ${gene_bed} -b ${introns_bed} | wc -l)
echo "${num_genes_no_introns} genes with no introns"
echo ""
echo "----------------------------"
echo ""

# Determine percentage of genes with introns.
# Sets "scale" to get desired number of decimal places.
percent_genes_w_introns=$(echo "scale=4;  1 - (${num_genes_no_introns} / ${num_genes})" | bc)


echo "${percent_genes_w_introns} of genes contain introns"

# Cleanup
rm ${gene_bed} ${gene_gff} ${introns_bed}

ls -l

34947 found in Panopea-generosa-v1.0.a4.gene.bed

----------------------------

9772 genes with no introns

----------------------------

.7204 of genes contain introns
total 0
